# Construcción del dataset

En la carpeta `assets` se encuentran dos archivos CSV que contienen información de tweets en Inglés con su respectiva clasificación de sentimiento. El objetivo de este notebook es construir un dataset que contenga la información de ambos archivos. Se seleccionaran de cada archivo 2000 tweets positivos, 2000 tweets negativos y 2000 neutral. Se traducirán los tweets al español y se almacenarán en un archivo CSV.

In [ ]:
# Instalación de librerías
%pip install -r '../requirements.txt'

In [ ]:
# Importación de librerías
import pandas as pd
from sklearn.model_selection import train_test_split

#### Dataset 1

In [ ]:
# Lectura dataset 1
header = ['id', 'entity', 'sentiment', 'text']
dataset1_no_procesado = pd.read_csv('./assets/dataset1-no-procesado.csv', delimiter=',', encoding='utf-8', names=header)
# Descrición dataset
print(dataset1_no_procesado.describe())
# Visualización primeros registros
dataset1_no_procesado.head()

In [ ]:
# Limpieza dataset 1
dataset1_procesado = dataset1_no_procesado.dropna()
dataset1_procesado = dataset1_procesado.drop_duplicates(subset='text')
dataset1_procesado = dataset1_procesado[dataset1_procesado['text'] != '']
dataset1_procesado = dataset1_procesado[(dataset1_procesado['sentiment'] == 'Positive') | 
                                        (dataset1_procesado['sentiment'] == 'Negative') | 
                                        (dataset1_procesado['sentiment'] == 'Neutral')]
# Descrición dataset
print(dataset1_procesado.describe())

In [ ]:
# Selección de columnas
dataset1_procesado = dataset1_procesado[['text', 'sentiment']]
dataset1_procesado.head()

In [ ]:
# Selección aleatoria de 5000 para cada clase
dataset1_procesado = dataset1_procesado.groupby('sentiment').apply(lambda x: x.sample(n=5000)).reset_index(drop=True)
# Descripción dataset
print(dataset1_procesado['sentiment'].value_counts())
print(dataset1_procesado['text'].head())

##### Dataset 2


In [ ]:
# Lectura dataset 2
dataset2_no_procesado = pd.read_csv('./assets/dataset2-no-procesado.csv', delimiter=',', encoding='unicode_escape')
# Descrición dataset
print(dataset2_no_procesado.describe())
# Visualización primeros registros
dataset2_no_procesado.head()

In [ ]:
# Limpieza dateset 2
dataset2_procesado = dataset2_no_procesado.dropna()
dataset2_procesado = dataset2_procesado[dataset2_procesado['selected_text'] != '']
dataset2_procesado = dataset2_procesado.drop_duplicates(subset='selected_text')
dataset2_procesado = dataset2_procesado[(dataset2_procesado['sentiment'] == 'positive') | 
                                        (dataset2_procesado['sentiment'] == 'negative') | 
                                        (dataset2_procesado['sentiment'] == 'neutral')]
# Descrición dataset
print(dataset1_procesado.describe())

In [ ]:
# Selección de columnas
dataset2_procesado = dataset2_procesado[['selected_text', 'sentiment']]
# Renombrar columnas
dataset2_procesado.columns = ['text', 'sentiment']
dataset2_procesado.head()

In [ ]:
# Selección aleatoria de 5000 para cada clase
dataset2_procesado = dataset2_procesado.groupby('sentiment').apply(lambda x: x.sample(n=5000)).reset_index(drop=True)
# Descripción dataset
print(dataset2_procesado['sentiment'].value_counts())
print(dataset2_procesado['text'].head())

# Juntar los dos datasets en un solo archivo CSV

In [ ]:
# Unión de datasets
dataset = pd.concat([dataset1_procesado, dataset2_procesado])
# Dejar clases en minúsculas
dataset['sentiment'] = dataset['sentiment'].str.lower()
# Descripción dataset
print(dataset['sentiment'].value_counts())

In [ ]:
# Exportación dataset 30000 registros
# train  = 80% = 24000 registros
# test = 20% = 6000 registros ->
dataset_train, dataset_other = train_test_split(dataset, test_size=0.2, random_state=42, stratify=dataset['sentiment'])
dataset_test, dataset_validation = train_test_split(dataset_other, test_size=0.5, random_state=42, stratify=dataset_other['sentiment'])
dataset_train.to_csv('./dist/dataset-english-tweets-train.csv', index=False, encoding='utf-8')
dataset_test.to_csv('./dist/dataset-english-tweets-test.csv', index=False, encoding='utf-8')
dataset_validation.to_csv('./dist/dataset-english-tweets-validation.csv', index=False, encoding='utf-8')

In [ ]:
print('Total registros:', dataset.shape[0])
print('Train registros:', dataset_train.shape[0])
print('Test registros:', dataset_test.shape[0])
print('Validation registros:', dataset_validation.shape[0])